# Camada trusted
## Criação de características relevantes

Importando biblotecas necessárias

In [123]:
import pandas as pd

lendo base de dados original com reclamações do consumidor.gov em formato csv.

In [101]:
claims = pd.read_csv(
    filepath_or_buffer='../dataset/raw/consumidor_gov.csv',
    delimiter=',',
    quotechar='"',
    header=0,
    names=[
        'company_name',
        'date',
        'uf',
        'city',
        'customer_claim',
        'company_response'
    ],
)

Convertendo coluna `date` do tipo _string_ para o tipo _date_.

In [102]:
claims['date'] = claims['date'].astype('datetime64[ns]')

Removendo caracteres especias (`\n`, `\r`, `\t`) de início e fim de texto para as colunas `customer_claim` e `company_response`.

In [104]:
claims['customer_claim'] = claims['customer_claim'].apply(lambda text: text.strip())
claims['company_response'] = claims['company_response'].apply(lambda text: text.strip())

Substituindo caracteres especiais (`\n`, `\r`, `\t`) no meio do texto por espaço vazio para as colunas `customer_claim` e `company_response`.

In [105]:
claims['customer_claim'] = claims['customer_claim'].str.replace(
    pat='[\n\r\t]+',
    repl=' ',
    regex=True
)

In [109]:
claims['company_response'] = claims['company_response'].str.replace(
    pat='[\n\r\t]+',
    repl=' ',
    regex=True
)

Explicitando reclamações sem resposta com valor `None`.

In [114]:
claims['company_response'] =  claims['company_response'].replace({'<não respondido pela empresa>': None})

Removendo índice original do arquivo csv.

In [119]:
claims.reset_index(drop=True, inplace=True)

Visualizando $5$ primeiros registros.

In [120]:
claims.head()

,company_name,date,uf,city,customer_claim,company_response
0,Correios,2021-04-13,MG,São João Del Rei,"Boa tarde ! No dia 08/04/2021, por volta das 1...","Prezado Cliente, para que possamos verificar o..."
1,Samsung,2021-04-25,MG,Belo Horizonte,Fui realizar essa ultima atualização do androi...,"Caro cliente, Conforme contato realizado atrav..."
2,Vivo - Telefônica,2021-04-14,MG,Caratinga,"Ao contatar a empresa, solicitei a 2ª via de c...",Prezada Sra. Vitoria Realizamos tentativa de c...
3,Vivo - Telefônica,2021-04-13,MG,Carmópolis de Minas,Fiz uma reclamação anteriormente a respeito de...,Prezada Sra. Juliana Realizamos tentativa de c...
4,Vivo - Telefônica,2021-04-14,MG,Santa Bárbara do Leste,Solicitei a Vivo plano para o número (33) XXXX...,Contatamos o responsável pela manifestação no ...


Analisando tipos de dados das colunas.

In [121]:
claims.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 378574 entries, 0 to 378573
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   company_name      378574 non-null  object        
 1   date              378574 non-null  datetime64[ns]
 2   uf                378574 non-null  object        
 3   city              378574 non-null  object        
 4   customer_claim    378574 non-null  object        
 5   company_response  375107 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 17.3+ MB


Salvando todo o conjunto de dados tratado e transformado em parquet.

In [124]:
claims.to_parquet(
    path='../dataset/trusted/claims.parquet.snappy',
    compression='snappy',
    index=True,
)

Salvando amostra do conjunto de dados tratado e transformado em parquet.

In [128]:
claims.sample(100).to_parquet(
    path='../dataset/trusted/sample_claims.parquet.snappy',
    compression='snappy',
    index=True,
)